In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

import edward as ed
ed.set_seed(42)

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from edward.models import Normal, Categorical, PointMass

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
N = 100   # number of images in a minibatch.
D = 784   # number of features.
K = 10    # number of classes.
epochs = 100
n_samples = 100
eval_batch_size = 1000
iters = int(mnist.train.num_examples/N*epochs)

In [4]:
X_val = mnist.validation.images
Y_val = np.argmax(mnist.validation.labels,axis=1)

In [5]:
def neural_network(X):
    h = tf.matmul(X, W_0) + b_0
    return h

# MODEL
with tf.name_scope("model"):
    W_0 = Normal(loc=tf.zeros([D, K]), scale=tf.ones([D, K]), name="W_0")
    b_0 = Normal(loc=tf.zeros(K), scale=tf.ones(K), name="b_0")

    X = tf.placeholder(tf.float32, [None, D], name="X")
    y = Categorical(neural_network(X), name="y")
    
# INFERENCE
with tf.name_scope("posterior"):
    with tf.name_scope("qW_0"):
        qW_0 = PointMass(tf.Variable(tf.random_normal([D, K]), name="loc"))
    with tf.name_scope("qb_0"):
        qb_0 = PointMass(tf.Variable(tf.random_normal([K]), name="loc"))
    

In [6]:
def eval_acc_auc(dataset):
    prob_lst = []
    pred_lst = []
    Y = []

    for _ in tqdm(range(dataset.num_examples//eval_batch_size)):
        X, y = mnist.test.next_batch(eval_batch_size, shuffle=False)
        prob_lst_temp = []
        for _ in range(n_samples):
            W0_samp = qW_0.sample()
            b0_samp = qb_0.sample()

            h_samp = tf.matmul(X, W0_samp) + b0_samp
            prob = tf.nn.softmax(h_samp)

            # Also compue the probabiliy of each class for each (w,b) sample.
            prob_lst_temp.append(prob.eval())
        pred_lst.append(np.argmax(np.mean(prob_lst_temp,axis=0),axis=1))
        prob_lst.append(np.max(np.mean(prob_lst_temp,axis=0),axis=1))
        Y.append(y)

    Y_pred = np.concatenate(pred_lst)
    Y_prob = np.concatenate(prob_lst)
    Y_actual = np.argmax(np.concatenate(Y), axis=1)

    acc = (Y_pred == Y_actual).mean()*100

    auc = roc_auc_score(Y_pred == Y_actual, Y_prob)

    return acc, auc

In [7]:
y_ph = tf.placeholder(tf.int32, [N])

inference = ed.MAP({W_0: qW_0, b_0: qb_0}, data={y: y_ph})

inference.initialize(n_iter=iters, n_print=100, scale={y: float(mnist.train.num_examples) / N})

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

for i in range(1, inference.n_iter+1):
    X_batch, Y_batch = mnist.train.next_batch(N)
    Y_batch = np.argmax(Y_batch,axis=1)
    info_dict = inference.update(feed_dict={X: X_batch, y_ph: Y_batch})
    inference.print_progress(info_dict)
    #if i*N % mnist.train.num_examples == 0: # epoch
    #    print('  Val acc:', eval_acc(X_val, Y_val))

55000/55000 [100%] ██████████████████████████████ Elapsed: 157s | Loss: 15719.592


In [8]:
X_test = mnist.test.images
Y_test = np.argmax(mnist.test.labels,axis=1)
print('MAP LR', epochs, n_samples, eval_acc_auc(mnist.test))

100%|██████████| 10/10 [05:42<00:00, 34.26s/it]

MAP LR 100 100 (92.090000000000003, 0.89080132267683498)
